In [1]:
#Libraries

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import re
import json
import string
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from tqdm.autonotebook import tqdm
from functools import partial
import torch
!pip install transformers
from transformers import BertTokenizer, BertModel
import spacy

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 901kB 36.9MB/s 
     |████████████████████████████████| 3.3MB 54.2MB/s 


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Wed Apr 28 19:38:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#data = pd.read_csv("data.csv")
data = pd.read_csv("data.csv")
data.head()
print(len(data))

4155


In [4]:
def read_GloVe(filename):
 embeddings = {}
 for line in open(filename).readlines():

    fields = line.strip().split(" ")
    word = fields[0]
    embeddings[word] = [float(x) for x in fields[1:]]
 return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

In [5]:
data.head()

,Unnamed: 0,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,mask
0,4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor...,True
1,5,81b5f874-9b42-4d4f-8c50-b4bab24f8aed,The Gender Gap Reloaded: Are School Characteri...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study examines the wage gender gap of yo...,True
2,6,f35fdfa6-a3f9-4ef3-a858-c8e2efcd2349,The High School Environment and the Gender Gap...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Despite the striking reversal of the gender g...,True
3,8,e5a4c6af-0854-4f7e-b088-06f03c18a01a,Locus of Control and Peer Relationships Among ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Abstract Past research has shown that locus o...,True
4,12,d5ff0ecd-ccf5-4f8a-a09c-182376eed4d0,Beyond the Laboratory: Evaluating the Survey E...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,"The black-white gap in achievement, as measur...",True


In [6]:
import re
paragraphs = []
for i in range(len(data)):
  t = data.loc[i][6]
  t = t.lower()
  t = re.findall(r"[\w']+|[.,!?;]", t)
  paragraphs.append(t)


In [7]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in paragraphs for w in l])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])


#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in paragraphs for w in l] + list(GloVe.keys())))}

i2word = {i:w for w,i in word2i.items()}




#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
  if train and (w in singletons and random.random() > 0.5):
    return 1
  else:
    return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
  #1.0 => UNK
  return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

X       = sentences2indices(paragraphs, word2i, train=True)


print(i2word[253])

#Print out some examples of what the dev inputs will look like
for i in range(10):
  print(" ".join([i2word.get(w,'UNK') for w in X[i]]))
def prepare_input(X_list):
  X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor)
  X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor)
  return (X_padded, X_mask)

marulis
this article investigates an important factor in student achievement parental involvement . using data from the national education longitudinal study nels , we estimate a value added education production function that includes parental effort as an input . parental effort equations are also estimated as a function of child , parent , household , and school characteristics . our results suggest that parental effort has a strong positive effect on achievement that is large relative to the effect of school resources and is not captured by family background variables . parents appear to reduce their effort in response to increased school resources , suggesting potential ''crowding out'' of school resources .
this study examines the wage gender gap of young adults in the 1970s , 1980s , and 2000 in the us . using quantile regression we estimate the gender gap across the entire wage distribution . we also study the importance of high school characteristics in predicting future labor 

In [8]:
import torch.nn as nn
num_words = len(word2i) + 2
weights = torch.Tensor(np.zeros((num_words, 300)))
E = nn.Embedding(num_words, 300)
for i in range(0,num_words):
  if i in i2word.keys():
    if i2word[i] in GloVe.keys():
      weights[i] = torch.Tensor(GloVe[i2word[i]])
    else:
      
      weights[i] = E.weight[i]
  else:
    weights[i] = E.weight[i]
E.weight.data = weights



In [9]:
gold_seq = []
count = 0
for i in range(len(data)):
  words = paragraphs[i]
  label = data.loc[i][4]
  t = label.lower()
  t = re.findall(r"[\w']+|[.,!?;]", t)
  seq = np.zeros(len(words))
  for j in range(len(words)):
    if words[j:j+len(t)] == t:
      index = (j, j+len(t))
      break
    else:
      index = False
  seq = np.zeros(len(words))
  if index:
    for j in range(index[0], index[1]):
      seq[j] = 1
  if all(v == 0 for v in seq):
    count+= 1
    for j in range(len(words)):
      for z in t:
        if words[j] in z:
          seq[j] = 1
          t.remove(z)
          break   
  gold_seq.append(seq)

    



In [10]:

class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()

        self.E = nn.Embedding(num_words, 300)
        self.init_glove(GloVe)
        self.L = nn.LSTM(DIM_EMB, DIM_HID, bidirectional=True, batch_first=True)
        self.W = nn.Linear(DIM_HID*2, 2)
        self.logSoftmax = nn.LogSoftmax(dim=2)
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax

    def forward(self, X, train=False):
        #TODO: Implement the forward computation.
        X = X.cuda()
        
        x = self.E(X)

        x, hidden = self.L(x)

        x = self.W(x)
        return self.logSoftmax(x)  

    def init_glove(self, GloVe):
      weights = torch.Tensor(np.zeros((num_words, 300)))
      for i in range(0,num_words):
        if i in i2word.keys():
          if i2word[i] in GloVe.keys():
            weights[i] = torch.Tensor(GloVe[i2word[i]])
          else:
            
            weights[i] = self.E.weight[i]
        else:
          weights[i] = self.E.weight[i]
      self.E.weight.data = weights
    def inference(self, sentences):

      pred = self.forward(sentences).argmax(dim=2)
      return pred


#The following code will initialize a model and test that your forward computation runs without errors.
lstm   = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm = lstm.cuda()
print(X[0])
lstm_output = lstm.forward(torch.LongTensor(X[0]).reshape((1,len(X[0])))).cuda()


#Check the shape of the lstm_output and one-hot label tensors.



[43864, 67965, 7832, 66689, 12325, 11515, 50548, 40974, 31024, 48933, 10550, 50319, 69259, 42497, 65289, 35347, 32453, 66685, 50354, 61569, 53847, 47309, 44876, 23181, 65043, 57913, 27971, 66685, 59965, 52553, 59159, 49246, 48933, 71558, 58334, 66689, 39539, 50319, 48933, 71558, 3388, 30718, 11232, 34424, 58334, 65043, 52553, 13429, 50854, 47309, 63205, 47309, 67929, 47309, 22361, 1186, 60168, 50319, 14866, 16278, 39327, 59159, 48933, 71558, 71610, 65043, 62668, 1006, 36358, 18148, 31024, 59159, 47160, 6423, 33089, 16007, 35347, 36358, 13429, 1186, 23347, 22361, 47160, 57645, 12796, 12044, 1396, 66884, 28827, 50319, 20084, 5420, 16007, 1862, 24133, 71558, 50548, 59024, 16007, 52164, 1186, 23347, 47309, 46859, 56526, 21657, 14446, 13429, 1186, 23347, 50319]


In [11]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
import torch.optim as optim
def shuffle_sentences(sentences, tags):
  shuffled_sentences = []
  shuffled_tags      = []
  indices = list(range(len(sentences)))
  random.shuffle(indices)
  for i in indices:
    #print(len(sentences[i]), len(tags[i]))
    shuffled_sentences.append(sentences[i])
    shuffled_tags.append(tags[i])
  return (shuffled_sentences, shuffled_tags)
nEpochs = 7

def train_basic_lstm(sentences, tags, lstm):
  optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer
  
  batchSize = 10
  for epoch in range(nEpochs):
    totalLoss = 0.0
    l = nn.CrossEntropyLoss(reduction="sum", ignore_index=10).cuda()
    (shuffled_sentences, shuffled_tags) = shuffle_sentences(sentences, tags)
    X       = sentences2indices(shuffled_sentences, word2i, train=True)
    for batch in tqdm.notebook.tqdm(range(0, len(sentences) - batchSize, batchSize), leave=False):
      lstm.zero_grad()
      #TODO: Impelement gradient update.
      
      input = prepare_input(X[batch:batch+batchSize-1])[0]
      Y = []
      for j in range(batch, batch+batchSize-1):
        Y.append(torch.LongTensor(shuffled_tags[j]))
      Y_padded = torch.nn.utils.rnn.pad_sequence(Y, batch_first=True,padding_value=10)
      lstm_output = lstm.forward(input)
      loss = l(torch.flatten(lstm_output, start_dim=0, end_dim = 1), torch.flatten(Y_padded).cuda())
      loss.backward()
      totalLoss += loss
      
      optimizer.step()
      torch.cuda.empty_cache()

    print(f"loss on epoch {epoch} = {totalLoss}")
    
lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_basic_lstm(paragraphs, gold_seq, lstm)

loss on epoch 0 = 73410.4375


loss on epoch 1 = 19581.6015625


loss on epoch 2 = 16927.537109375


loss on epoch 3 = 15610.2236328125


loss on epoch 4 = 14828.1796875


loss on epoch 5 = 11705.6982421875


loss on epoch 6 = 10439.1142578125


In [ ]:
torch.save(lstm.state_dict(), "lstm.pt")

In [12]:
for j in range(0,len(data), 50):
  z = torch.LongTensor(X[j])

  result = lstm.inference(z.reshape((1,len(X[j])))).tolist()
  result = np.array(result[0])
  index = np.where(result == 1)[0]
  print(index)
  for i in index:
    print(i2word[X[j][i]])
  print(data.loc[j])


[16 17 18 19]
national
education
longitudinal
study
Unnamed: 0                                                       4
Id                            c754dec7-c5a3-4337-9892-c02158475064
pub_title        Parental Effort, School Resources, and Student...
dataset_title                National Education Longitudinal Study
dataset_label                National Education Longitudinal Study
cleaned_label                national education longitudinal study
text              This article investigates an important factor...
mask                                                          True
Name: 0, dtype: object
[817 818]
slosh
sea
Unnamed: 0                                                     347
Id                            07ce992d-e2f3-49cb-bedb-49616a19405f
pub_title        U.S. IOOS coastal and ocean modeling testbed: ...
dataset_title       Sea, Lake, and Overland Surges from Hurricanes
dataset_label                                          SLOSH model
cleaned_label                     

In [ ]:
from google.colab import files
files.download("lstm.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Stopped here

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')



In [ ]:
tokens = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
#filtered = []
#for d in range(len(data)):
 # sample = data.loc[d][6]
#  t = tokens(sample)["input_ids"][1:-1]
  #chunks = []
#  for i in range(0, len(t), 500):
   # chunks.append(tokens.decode(t[i:i+500]))
 # j = 0
 # for i in range(len(chunks)):
  #  if data.loc[d][5] in chunks[i]:
   #   j = i
   #   break
  #filtered.append(chunks[j])


In [ ]:
#data["filtered"] = filtered
#data.to_csv("data_filtered.csv")


In [ ]:
data = pd.read_csv("data_filtered.csv")

Crashes here trying to find hidden states

In [ ]:
states = []
for i in range(len(data)):  
  if i /50 == 0:
    print(i)

  t = tokens(data.loc[i][-1], return_tensors="pt")

  states.append(model(**t).last_hidden_state)

In [ ]:
sample = data.loc[0][6]
count = 0
print(len(t))
token_col = []
len_col = []
for i in range(len(data)):
  t = tokens(data.loc[i][6], return_tensors="pt")
  token_col.append(t)
  len_col.append(t["input_ids"].shape)
  if t["input_ids"].shape[1] > 512:
    count +=1
print(count)
print(sample)
t = tokens(sample, return_tensors="pt", max_length=512)
print(t["input_ids"].shape[1])
outputs = model(**t)
last_hidden_layers = outputs.last_hidden_state
print(last_hidden_layers.shape)

In [ ]:

changed_text = []
t = tokens(data.loc[0][6])
print(tokens.decode(t["input_ids"]))
print(data.loc[0][6])
for i in range(len(data)):
  t = data.loc[i][6].split(' ')

In [ ]:
print(len_col)

In [ ]:
data = data[data["token_len"] < 512]
print(len(data))

In [ ]:
from spacy.lang.en import English
spacy_sent = spacy.load('en_core_web_sm')
sentences = spacy_sent(data.loc[0][6]).sents
base = torch.zeros((1,0,768))
print(data.loc[0][6])
for i in sentences:
   print(i.text)
   #t = tokens(i.text, return_tensors="pt")
   #outputs = model(**t)
   #last_hidden_layers = outputs.last_hidden_state
   #base = torch.cat((base, last_hidden_layers), dim=1)


In [ ]:
print(base.shape)

In [ ]:
t = tokens(data.loc[0][6], return_tensors="pt")
print(t["input_ids"].shape)